In [85]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import linregress
import cpi
cpi.update()

In [86]:
# CSV Path
imdb_movies_csv = "Data/imdb_movies.csv"
box_office_csv = "Data/boxoffice.csv"

In [87]:
# Variables for the seperate data frames 
imdb_df = pd.read_csv(imdb_movies_csv)
box_office_df = pd.read_csv(box_office_csv)

In [88]:
# Removing columns I do not need
col_imdb_df = imdb_df[['imdb_title_id', 'original_title', 'year', 'date_published', 'genre', 'duration', 'country', 
                         'language', 'director', 'writer', 'production_company', 'actors', 'avg_vote', 'votes']]

col_box_office_df = box_office_df[['title', 'lifetime_gross', 'year']]


In [89]:
# Renaming columns to be more uniform
col_imdb_df = col_imdb_df.rename(columns={'imdb_title_id': "IMDB ID", 'original_title': 'Title', 'year': 'Year', 
                                          'date_published': 'Date Published', 'genre': 'Genre', 'duration': 'Duration',
                                         'country': 'Country', 'language': 'Language', 'director': 'Director', 'writer': 'Writer',
                                         'production_company': 'Production Company', 'actors': 'Actors', 'avg_vote': 'Avg Rating',
                                          'votes': '# of Ratings'})

col_box_office_df = col_box_office_df.rename(columns={'title': 'Title', 'lifetime_gross': 'Lifetime_Gross', 'year': 'Year'})

In [90]:
# Merging data frames
movie_data = pd.merge(col_imdb_df, col_box_office_df, on=['Title', 'Year'], how='inner')

# Dropping data that is not available through CPI
movie_data.drop(movie_data[movie_data['Year'] == 1912 ].index , inplace=True)
movie_data.drop(movie_data[movie_data['Year'] == 2019 ].index , inplace=True)
movie_data.drop(movie_data[movie_data['Year'] == 2020 ].index , inplace=True)

In [91]:
# Sort IMDB ID
movie_data.sort_values("IMDB ID", inplace = True)

# Drop duplicate IMDB ID
movie_data.drop_duplicates(subset ="IMDB ID", keep = False, inplace = True)

In [92]:
# Calculating inflation through CPI library 
# Source: https://medium.com/analytics-vidhya/adjusting-for-inflation-when-analysing-historical-data-with-python-9d69a8dcbc27
def inflate_column(data, column):
    
    return data.apply(lambda x: cpi.inflate(x[column], x.Year), axis=1)          
    
movie_data['Lifetime Gross - Adjusted ($)'] = inflate_column(movie_data, 'Lifetime_Gross')

In [93]:
# Converting scientific notation
movie_data["Lifetime Gross - Adjusted ($)"].describe().apply(lambda x: format(x, 'f'))
pd.options.display.float_format = '{:,.2f}'.format

# Formatting columns
movie_data['Lifetime_Gross'] = movie_data['Lifetime_Gross'].astype(float).map("${:,.2f}".format)
movie_data['Lifetime Gross - Adjusted ($)'] = movie_data['Lifetime Gross - Adjusted ($)'].astype(float).map("${:,.2f}".format)

In [95]:
movie_data.head()

,IMDB ID,Title,Year,Date Published,Genre,Duration,Country,Language,Director,Writer,Production Company,Actors,Avg Rating,# of Ratings,Lifetime_Gross,Lifetime Gross - Adjusted ($)
0,tt0012190,The Four Horsemen of the Apocalypse,1921,1923-01-08,"Drama, Romance, War",150,USA,None,Rex Ingram,"Vicente Blasco Ibáñez, June Mathis",Metro Pictures Corporation,"Pomeroy Cannon, Josef Swickard, Bridgetta Clar...",7.20,2935,"$9,183,673.00","$128,831,540.56"
1,tt0017136,Metropolis,1927,1927-02-06,"Drama, Sci-Fi",153,Germany,German,Fritz Lang,"Thea von Harbou, Thea von Harbou",Universum Film (UFA),"Alfred Abel, Gustav Fröhlich, Rudolf Klein-Rog...",8.30,148396,"$1,236,166.00","$17,839,651.48"
2,tt0017354,The Sea Beast,1926,1926-01-15,"Action, Adventure, Fantasy",136,USA,English,Millard Webb,"Herman Melville, Bess Meredyth",Warner Bros.,"John Barrymore, Dolores Costello, George O'Har...",6.50,126,"$814,000.00","$11,548,084.63"
3,tt0021749,City Lights,1931,1931-08-21,"Comedy, Drama, Romance",87,USA,English,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Virginia Cherrill, Florence Lee, Harry Myers, ...",8.50,152716,"$19,181.00","$316,873.91"
4,tt0027977,Modern Times,1936,1936-10-16,"Comedy, Drama, Family",87,USA,English,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Charles Chaplin, Paulette Goddard, Henry Bergm...",8.50,197969,"$163,577.00","$2,955,059.69"


In [97]:
# Pushing the cleaned DF to a CSV file
movie_data.to_csv("Data/data_cleaning.csv")